In this file we focus on generating more features - but this time with a focus on time series. We first load the relevant packages.  

In [1]:
import numpy as np 
import pandas as pd

We then read the input files and display the first few records to read the format. Note that we are using the calendar data with the new features we created - instead of using the format given to us.

In [2]:
sell_prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')

In [3]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [4]:
sales_data = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')

In [5]:
sales_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [8]:
calendar = pd.read_csv('/kaggle/input/calendar/Calendar_Data_Clean.csv')

In [9]:
calendar.head()

,wm_yr_wk,month,d,snap_CA,snap_TX,snap_WI,Religious_Holiday,Cultural_Holiday,Sporting_Holiday,National_Holiday,weekend,Month_End,Month_Start,Quarter,High_Spending_Month,Low_Spending_Month
0,11101,1,d_1,0,0,0,0,0,0,0,1,1,0,1,0,0
1,11101,1,d_2,0,0,0,0,0,0,0,1,1,0,1,0,0
2,11101,1,d_3,0,0,0,0,0,0,0,0,1,0,1,0,0
3,11101,2,d_4,1,1,0,0,0,0,0,0,0,1,1,0,0
4,11101,2,d_5,1,0,1,0,0,0,0,0,0,1,1,0,0


We use a downcast function. This is very useful to help ensure that we do not use more RAM - and objects where possible take the minimum amount of space required in our workflow. 

In [10]:
def downcast(df):
    
    cols = df.dtypes.index.tolist()
    types = df.dtypes.values.tolist()

    for i,t in enumerate(types):

        if 'int' in str(t):
            if df[cols[i]].min() > np.iinfo(np.int8).min and df[cols[i]].max() < np.iinfo(np.int8).max:
                df[cols[i]] = df[cols[i]].astype(np.int8)
            elif df[cols[i]].min() > np.iinfo(np.int16).min and df[cols[i]].max() < np.iinfo(np.int16).max:
                df[cols[i]] = df[cols[i]].astype(np.int16)
            elif df[cols[i]].min() > np.iinfo(np.int32).min and df[cols[i]].max() < np.iinfo(np.int32).max:
                df[cols[i]] = df[cols[i]].astype(np.int32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.int64)

        elif 'float' in str(t):
            if df[cols[i]].min() > np.finfo(np.float16).min and df[cols[i]].max() < np.finfo(np.float16).max:
                df[cols[i]] = df[cols[i]].astype(np.float16)
            elif df[cols[i]].min() > np.finfo(np.float32).min and df[cols[i]].max() < np.finfo(np.float32).max:
                df[cols[i]] = df[cols[i]].astype(np.float32)
            else:
                df[cols[i]] = df[cols[i]].astype(np.float64)

        elif t == object:
            if cols[i] == 'date':
                df[cols[i]] = pd.to_datetime(df[cols[i]], format='%Y-%m-%d')
            else:
                df[cols[i]] = df[cols[i]].astype('category')

    return df  

We use the above function across all our input files

In [11]:
sell_prices = downcast(sell_prices)

In [12]:
calendar = downcast(calendar)

We notice from our time series that there is a lot of variation across the first few records. Further we also faced issues with both performance in terms of results as well as computing when we consider all the records - hence we drop the first 250 days in our process. To do this we consider to create a list of all values as we need ranging from 1 to 250

In [13]:
d_li = []
for i in range(1,250):
    d_li.append('d_' + str(i))

We drop these columns

In [14]:
sales_data = sales_data.drop(columns = d_li,axis = 1)

We now add the columns we need from 1942-1970 days and impute them with a value of 0. These will be the values from our evaluation phase - and treated as unknown.

In [15]:
for d in range(1942,1970):
    col = 'd_' + str(d)
    sales_data[col] = 0

In [16]:
sales_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_250,d_251,d_252,d_253,...,d_1960,d_1961,d_1962,d_1963,d_1964,d_1965,d_1966,d_1967,d_1968,d_1969
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,4,2,2,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,1,1,3,...,0,0,0,0,0,0,0,0,0,0


In [17]:
sales_data = downcast(sales_data)

We can now melt or pivot down the data at day level - considering all the features we have.

In [18]:
sales_data =pd.melt(sales_data,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name='d',value_name='demand')

In [19]:
sales_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_250,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_250,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_250,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_250,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_250,0


We now focus on creating time series features below. We update the datatype of days to int by removing the prefix in the column earlier - now that we have it at row level.

In [20]:
sales_data['d'] = sales_data['d'].str.replace('d_','').astype('int')

We consider creating lag features for 28-30 days. We also create two sets of rolling window features - with a window of size 7 and 28 for values of 28 and 30 days. 

Lag features are values at prior timesteps that are considered useful because they are created on the assumption that what happened in the past can influence or contain a sort of intrinsic information about the future.

In the case of a rolling window, the size of the window is constant while the window slides as we move forward in time.The idea behind the expanding window feature is that it takes all the past values into account.

In [21]:
sales_data['lag_28'] = sales_data.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
sales_data['lag_29'] = sales_data.groupby(['id'])['demand'].transform(lambda x: x.shift(29))
sales_data['lag_30'] = sales_data.groupby(['id'])['demand'].transform(lambda x: x.shift(30))
sales_data['rolling_mean_7_28']   = sales_data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
sales_data['rolling_std_7_28']    = sales_data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
sales_data['rolling_mean_30_28']  = sales_data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).mean())
sales_data['rolling_std_30_28']  = sales_data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(30).std())

We follow the same for rolling means 90 and 180 days with 28 days as the size of the window

In [22]:
sales_data['rolling_mean_90_28']  = sales_data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(90).mean())
sales_data['rolling_mean_180_28'] = sales_data.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(180).mean())

In [23]:
sales_data = downcast(sales_data)

We now update our calendar data - as we try to merge with the dataframe containing the sales value above. The first step is updating the column containing days as seen earlier in our sales dataframe.

We then merge on the same column. We do not write into a new dataframe to enable us to save memory.

In [24]:
calendar['d'] = calendar['d'].str.replace('d_','').astype('int')

In [25]:
sales_data = pd.merge(sales_data,calendar, on = ['d'],how = 'left')

In [26]:
sales_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,lag_28,lag_29,...,Religious_Holiday,Cultural_Holiday,Sporting_Holiday,National_Holiday,weekend,Month_End,Month_Start,Quarter,High_Spending_Month,Low_Spending_Month
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,0,0,0,0,1,4,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,0,0,0,0,1,4,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,0,0,0,0,1,4,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,0,0,0,0,1,4,0,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,0,0,0,0,1,4,0,0


We then merge on the selling prices as well. This time we decide to merge on an extended set of keys.

In [27]:
sales_data = pd.merge(sales_data,sell_prices,on = ['item_id','store_id','wm_yr_wk'],how = 'left')

In [28]:
sales_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,lag_28,lag_29,...,Cultural_Holiday,Sporting_Holiday,National_Holiday,weekend,Month_End,Month_Start,Quarter,High_Spending_Month,Low_Spending_Month,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,0,0,0,1,4,0,0,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,0,0,0,1,4,0,0,3.970703
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,0,0,0,1,4,0,0,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,0,0,0,1,4,0,0,4.339844
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,0,0,0,1,4,0,0,2.480469


We also noticed that there were a number of missing values in the sell prices file. To overcome this we want to impute the values with the mean. However, taking a strict mean across the column would not be correct as we have a lot of variation in price as well. 

Hence we decide to take mean at item and store level across the weeks. This will act as the level where we impute data for our prices

In [29]:
mean_prices = sell_prices.groupby(['item_id','store_id'])['sell_price'].mean().reset_index()

We again merge the data - this time forcing it on the same index as the level we had created the data. We then follow it up by updating the NA's with the same value in the row with the mean value. 

In [30]:
sales_data = pd.merge(sales_data,mean_prices,on = ['item_id','store_id'],how = 'left')

In [31]:
sales_data['sell_price_x'].fillna(sales_data['sell_price_y'],inplace = True)

In [32]:
sales_data.drop(columns = ['sell_price_y','wm_yr_wk'],inplace = True)

One interesting feature we came across during our analysis is the method to incorporate - change over time. We decided to use the same. 

The reasoning behind incorporating the change would be that there are variations in price which can cause change in demand as well - both price and demand go hand in hand. Hence it would be ideal to consider a situation where we include this change as a feature too - similar to the feature included for demand lags above

We extend to include mean and standard deviation as considering only mean might affect outliers. We have taken the values at one week and one year for the same.

In [33]:
sales_data['lag_price_1'] = sales_data.groupby(['id'])['sell_price_x'].transform(lambda x: x.shift(7))
sales_data['price_change_1'] = (sales_data['lag_price_1'] - sales_data['sell_price_x']) / (sales_data['lag_price_1'])

In [34]:
sales_data['rolling_price_max_365_1'] = sales_data.groupby(['id'])['sell_price_x'].transform(lambda x: x.shift(1).rolling(365).max())
sales_data['price_change_365'] = (sales_data['rolling_price_max_365_1'] - sales_data['sell_price_x']) / (sales_data['rolling_price_max_365_1'])

In [35]:
sales_data['rolling_price_std_t7'] = sales_data.groupby(['id'])['sell_price_x'].transform(lambda x: x.rolling(7).std())
sales_data['rolling_price_std_t30'] = sales_data.groupby(['id'])['sell_price_x'].transform(lambda x: x.rolling(30).std())
sales_data.drop(['rolling_price_max_365_1', 'lag_price_1'], inplace = True, axis = 1)

As our final step - we decided to consider change in revenue as well. By considering both the change in price and revenue over time - we can get a better estimate of the demand too. 

The method to create the features is exactly the same as the ones we saw earlier. We extend to include mean and standard deviation here as well.

In [37]:
sales_data['revenue'] = sales_data['demand'] * sales_data['sell_price_x']
sales_data['lag_revenue_t1'] = sales_data.groupby(['id'])['revenue'].transform(lambda x: x.shift(28))
sales_data['rolling_revenue_std_t28'] = sales_data.groupby(['id'])['lag_revenue_t1'].transform(lambda x: x.rolling(28).std())
sales_data['rolling_revenue_mean_t28'] = sales_data.groupby(['id'])['lag_revenue_t1'].transform(lambda x: x.rolling(28).mean())
sales_data.drop(['revenue'],axis=1,inplace=True)

Once we have all the features - we write them into a pickle file. This will take up less space and we can experiment with our features and modelling - and spend less time running this notebook in the future.

In [38]:
sales_data.to_pickle('final_data.pkl')

In [39]:
sales_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,lag_28,lag_29,...,High_Spending_Month,Low_Spending_Month,sell_price_x,price_change_1,price_change_365,rolling_price_std_t7,rolling_price_std_t30,lag_revenue_t1,rolling_revenue_std_t28,rolling_revenue_mean_t28
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,8.281250,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,3.970703,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,2.970703,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,4.339844,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,250,0,NaN,NaN,...,0,0,2.480469,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Generating a link to download in Kaggle. 

In [40]:
import os
os.chdir(r'/kaggle/working')

In [41]:
from IPython.display import FileLink
FileLink(r'final_data.pkl')

/kaggle/working/final_data.pkl